In [4]:
import cv2
import numpy as np
from tqdm.auto import tqdm


# convert video to images
def convert_video_to_images(path_to_video, path_to_images, n_frames=100):
    # parameters:
    #   path_to_video - absolute or relative path to video file
    #   path_to_images - absolute or relative path to result frames folder
    #   n_frames - number of frames to sample
    cap = cv2.VideoCapture(path_to_video)
    count = 0
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_list = np.linspace(0, total_frames, n_frames, dtype=int)
    # folder = video_path.split('.')[0]
    frame_count = 0
    print("Video info")
    print("\tvideo fps:", fps)
    print("\tvideo total frames:", total_frames)
    print("\tvideo frames selected:", n_frames)
    pbar = tqdm(total=total_frames)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count in frame_list:
            cv2.imwrite(f"{path_to_images}/frame{str(frame_count).zfill(6)}.jpg", frame)
            frame_count += 1
        count += 1
        pbar.update(1)
    pbar.close()
    cap.release()
    cv2.destroyAllWindows()


path_to_video = "<PATH TO VIDEO>"
path_to_images = "<PATH TO IMAGES>"
convert_video_to_images(path_to_video, path_to_images)

Video info
	video fps: 30
	video total frames: 199
	video frames selected: 100


  0%|          | 0/199 [00:00<?, ?it/s]

In [2]:
import os
import cv2
from tqdm.auto import tqdm


# convert images to video
def convert_images_to_video(path_to_images, path_to_video):
    fc = cv2.VideoWriter_fourcc(*"mp4v")
    video = cv2.VideoWriter(path_to_video, fc, 30, (2000, 1109))
    for file in tqdm(
        sorted(os.listdir(path_to_images)), desc="Convertion in progress.."
    ):
        file_path = os.path.join(path_to_images, file)
        image = cv2.imread(file_path)
        video.write(image)


path_to_images = "<PATH TO IMAGES>"
path_to_video = "<PATH TO VIDEO>"
convert_images_to_video(path_to_images, path_to_video)

Convertion in progress..:   0%|          | 0/199 [00:00<?, ?it/s]

In [5]:
from ultralytics import YOLO
from datetime import datetime, timezone
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm


def gen_cur_time():
    return datetime.now(timezone.utc).strftime("%Y-%m-%d %H-%M-%S-%f")


def yolov8_predict(model_name, path_to_images, classes, device, save):
    run_name = f"run-{gen_cur_time()}"
    model = YOLO(model_name)
    results = model.predict(
        path_to_images,
        save=save,
        imgsz=640,
        classes=classes,
        device=device,
        project="predict-results",
        name=run_name,
    )
    return results


def create_masks(results, path_to_masks):
    n_frames = len(results)
    n_unmasked_frames = 0
    for image_res in tqdm(results, desc="Yolov8 masks processing.."):
        pixel_masks = []
        for obj_res in image_res:
            pixel_masks.append(obj_res.masks.xy[0].astype(np.int32))
        img = np.zeros(image_res.orig_shape)
        for mask in pixel_masks:
            cv2.fillPoly(img, [mask], 255)
        # process frames with 0 detected objects
        if len(pixel_masks) == 0:
            img.fill(255)
            n_unmasked_frames += 1
        split_path = image_res.path.split("\\")
        if len(split_path) == 1:
            split_path = split_path[0].split("/")
        mask_filename = split_path[-1]
        mask_path = path_to_masks + "\\" + mask_filename + ".png"
        cv2.imwrite(mask_path, img)
    return n_frames - n_unmasked_frames, n_frames


def main(path_to_images, path_to_masks, classes, device, save):
    model_name = "object-detection/yolov8s-seg-co3d.pt"
    path_to_images = r"C:\Users\marem\dev\projects\unn\cw\current_year_run\final\runs\teddybear\all_images"
    classes = None
    device = "cuda:0"  # "cpu"
    save = True
    results = yolov8_predict(model_name, path_to_images, classes, device, save)
    n_masked_frames, n_frames = create_masks(results, path_to_masks)
    return n_masked_frames, n_frames, results


path_to_images = "<PATH TO IMAGES>"
path_to_masks = "<PATH TO MASKS>"
n_masked_frames, n_frames, results = main(
    path_to_images, path_to_masks, None, "cuda:0", False
)
print(f"All frames: {n_frames}\nFrames with instance masks: {n_masked_frames}")


image 1/202 C:\Users\marem\dev\projects\unn\cw\current_year_run\final\runs\teddybear\all_images\frame000001.jpg: 640x384 (no detections), 243.1ms
image 2/202 C:\Users\marem\dev\projects\unn\cw\current_year_run\final\runs\teddybear\all_images\frame000002.jpg: 640x384 (no detections), 13.9ms
image 3/202 C:\Users\marem\dev\projects\unn\cw\current_year_run\final\runs\teddybear\all_images\frame000003.jpg: 640x384 (no detections), 14.0ms
image 4/202 C:\Users\marem\dev\projects\unn\cw\current_year_run\final\runs\teddybear\all_images\frame000004.jpg: 640x384 1 teddy bear, 13.0ms
image 5/202 C:\Users\marem\dev\projects\unn\cw\current_year_run\final\runs\teddybear\all_images\frame000005.jpg: 640x384 1 teddy bear, 14.9ms
image 6/202 C:\Users\marem\dev\projects\unn\cw\current_year_run\final\runs\teddybear\all_images\frame000006.jpg: 640x384 1 teddy bear, 14.4ms
image 7/202 C:\Users\marem\dev\projects\unn\cw\current_year_run\final\runs\teddybear\all_images\frame000007.jpg: 640x384 1 teddy bear, 15

Yolov8 masks processing..:   0%|          | 0/202 [00:00<?, ?it/s]

All frames: 202
Frames with instance masks: 194
